# **Project 1: Data Analysis for Medical Applications**

---

### **Names:**
**Andrés Martínez Almazán A01621042** and **Diego Arechiga Bonilla A01621045**

### **Course:**
Modeling Learning with Artificial Intelligence

### **Professor:**
**Dr. Omar Mendoza Montoya**

### **Date:**
June 4, 2025

---

# Data Collection

### Selected Signal: **Audio Amplitude**

#### Experiment Design:
1. Silence/ambient (background noise)
2. Normal speaking (conversation)
3. Whispering
4. Shouting (loud voice)
5. Playing music (high volume)
6. Applause

### Collection Protocol:

For each activity:
- **Duration:** 40-60 seconds per recording.
- **Repetitions:** 5 repetitions per activity.
- **Conditions:** Perform in a controlled environment, without external noise.
- **Labeling:** Each recording must have a clear label indicating the activity performed.

### Data Collection Protocol:
1. Start recording in PhyPhox.
2. Perform the activity for the established time.
3. Stop recording and save the file.
4. Label the file with the activity name and repetition number.
5. Repeat the process for each activity.

### Hypothesis:
The audio amplitude will be different between each activity, thus allowing automatic classification of the type of sound in the environment.

In [ ]:
# imports 
import pickle
import numpy as np
from scipy import stats
import pandas as pd

In [153]:
actividades = [
    ('sonido_amb', 1),
    ('conversacion', 2), 
    ('susurro', 3),
    ('grito', 4),
    ('music', 5),
    ('aplausos', 6)
]

experiment_data = [] 

In [ ]:
# load files 

for actividad_nombre, actividad_id in actividades:
    print("Processing activity:", actividad_nombre)
    
    # load each test file 
    for i in range(1,6):
        nombre_archivo = f"../data/{actividad_nombre}_test{i}.csv"
        
        try:
            df = pd.read_csv(nombre_archivo)
            
            # extract audio signal (sound pressure level)
            if "Sound pressure level (dB)" in df.columns:
                signal_data = df["Sound pressure level (dB)"].values
                signal_2d = signal_data.reshape(-1, 1) # convert to 2D to adapt the audio signal
                
                experiment_data.append((actividad_nombre, actividad_id, signal_2d))
                print(f"{nombre_archivo}: {len(signal_data)} valid samples")
            
            else:
                print(f"{nombre_archivo}: Column 'Sound pressure level (dB)' not found")
                
        except FileNotFoundError:
            print(f"{nombre_archivo}: File not found")
        except Exception as e:
            print(f"{nombre_archivo}: Error - {e}")
                
print(f"\n=== SUMMARY ===")
print(f"Total files loaded: {len(experiment_data)}")
print(f"Structure created successfully")

# Evaluate classification algorithms

1. Process the data so that each observation has at least 30 variables or characteristics.

In [ ]:
# process data

print ("\n=== PROCESSING DATA ===")

features = []
for tr in experiment_data:
    
    # for each signal 
    feat = [tr[1]]  # actividad_id
    rms = 0 # Root Mean Square
    
    for s in range(tr[2].shape[1]):
        sig = tr[2][:, s] # extract the audio signal
        
        # filter NaN values
        sig = sig[~np.isnan(sig)]
        
        if len(sig) > 0: # only process if there is valid data
            feat.append(np.mean(sig))  # mean
            feat.append(np.std(sig)) # standard deviation
            feat.append(stats.skew(sig))  # skewness
            feat.append(stats.kurtosis(sig)) # kurtosis
            rms += np.sum(sig**2)  # sum of squares
            feat.append(np.median(sig))       # median
            feat.append(np.min(sig))          # minimum
            feat.append(np.max(sig))          # maximum
            feat.append(np.ptp(sig))          # Range (peak-to-peak)
            feat.append(np.var(sig))          # variance
            feat.append(np.percentile(sig, 10))   # Q10
            feat.append(np.percentile(sig, 25))   # Q25
            feat.append(np.percentile(sig, 75))   # Q75
            feat.append(np.percentile(sig, 90))   # Q90
            feat.append(np.percentile(sig, 75) - np.percentile(sig, 25))  # IQR
            feat.append(np.sum(sig**2))       # Total energy
            feat.append(np.mean(sig**2))      # Average power
            feat.append(np.sum(np.abs(sig)))  # Sum of absolute values
            
            if len(sig) > 1: # calculate velocity and acceleration
                velocity = np.diff(sig)       # first derivative
                feat.append(np.mean(velocity))    # Average velocity
                feat.append(np.std(velocity))     # Velocity deviation
                
                if len(velocity) > 1:
                    acceleration = np.diff(velocity)  # second derivative
                    feat.append(np.mean(acceleration))  # average acceleration
                    feat.append(np.std(acceleration))   # acceleration deviation
                else:
                    feat.extend([0, 0])
            else:
                feat.extend([0, 0, 0, 0])
            
            # Zero crossing rate (ZCR)
            # Calculate zero crossing rate
            mean_level = np.mean(sig)
            zero_crossings = np.sum(np.diff(np.signbit(sig - mean_level)))
            feat.append(zero_crossings / len(sig))  # zero crossing rate
            
            
            from scipy import stats as scipy_stats
            feat.append(scipy_stats.moment(sig, moment=3))  # Third moment
            feat.append(scipy_stats.moment(sig, moment=4))  # Fourth moment
            feat.append(np.mean(np.abs(sig - np.median(sig))))  # Median absolute deviation
            feat.append(len(sig))  # Number of samples
            
            # Characteristics
            feat.append(np.max(sig) / np.std(sig))  # Approximate SNR
            feat.append(np.percentile(sig, 90) - np.percentile(sig, 10))  # Dynamic range
            
            # Spectral centroid (temporal approximation)
            if np.sum(np.abs(sig)) > 0:
                weighted_sum = np.sum(np.abs(sig) * np.arange(len(sig)))
                feat.append(weighted_sum / np.sum(np.abs(sig)))  # Spectral centroid
            else:
                feat.append(0)
            
            # Spectral flux (average change)
            if len(sig) > 1:
                feat.append(np.mean(np.abs(np.diff(sig))))  # Spectral flux
            else:
                feat.append(0)
                
        else:
            # If there is no valid data, add zeros
            feat.extend([0] * 29)  # 29 additional characteristics + the original 4 = 33 total    
            
        
        
            
    rms = np.sqrt(rms)
    feat.append(rms)  # RMS (final characteristic)
    features.append(feat)
    
print(f"Features extracted per sample: {len(features[0])-1}")  # -1 because the first element is the ID
print(f"Total samples processed: {len(features)}")

# Build x and y arrays 
processed_data = np.array(features)
x = processed_data[:,1:]  # Features
y = processed_data[:,0]   # Labels

print(f"\nShape of X (features): {x.shape}")
print(f"Shape of y (labels): {y.shape}")
print(f"Unique labels: {np.unique(y)}")

2. Evaluate the performance of classification models:
   - **SVM**
   - **SVM with radial basis**
   - **LDA**
   - **K-NN**
    - **MLP**

In [ ]:
# Imports
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_predict, LeaveOneOut, RepeatedKFold, cross_val_score

In [157]:
processed_data_path = "../data/processed/audio_features.txt" 
processed_data= np.loadtxt(processed_data_path)

X = processed_data[:, 1:]  # Características
y = processed_data[:, 0]    # Labels

pipeline = make_pipeline(StandardScaler())
X_scaled = pipeline.fit_transform(X)  

Support Vector Machine evaluation:

In [158]:
n_folds = 5
clf = SVC(kernel = 'linear')
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         5
         2.0       0.57      0.80      0.67         5
         3.0       1.00      1.00      1.00         5
         4.0       0.60      0.60      0.60         5
         5.0       0.67      0.40      0.50         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.80        30
   macro avg       0.81      0.80      0.79        30
weighted avg       0.81      0.80      0.79        30



Support Vector Machine with radial base evaluation:

In [159]:
n_folds = 5
clf = SVC(kernel = 'rbf', random_state=42)
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         5
         2.0       0.57      0.80      0.67         5
         3.0       1.00      1.00      1.00         5
         4.0       0.60      0.60      0.60         5
         5.0       1.00      0.60      0.75         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.83        30
   macro avg       0.86      0.83      0.84        30
weighted avg       0.86      0.83      0.84        30



LDA evaluation:

In [160]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

n_folds = 5
clf = LinearDiscriminantAnalysis()
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      0.80      0.89         5
         2.0       0.75      0.60      0.67         5
         3.0       0.83      1.00      0.91         5
         4.0       0.71      1.00      0.83         5
         5.0       0.75      0.60      0.67         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.83        30
   macro avg       0.84      0.83      0.83        30
weighted avg       0.84      0.83      0.83        30



k-Nearest Neighbors evaluation:

In [161]:
from sklearn.neighbors import KNeighborsClassifier

n_folds = 5
clf = KNeighborsClassifier(n_neighbors=3)
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         5
         2.0       0.56      1.00      0.71         5
         3.0       1.00      1.00      1.00         5
         4.0       0.67      0.40      0.50         5
         5.0       1.00      0.60      0.75         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.83        30
   macro avg       0.87      0.83      0.83        30
weighted avg       0.87      0.83      0.83        30



Multi-layer perceptron evaluation:

In [162]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

n_folds = 5
clf = MLPClassifier(hidden_layer_sizes=(100), max_iter=10000, random_state=42)
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         5
         2.0       0.57      0.80      0.67         5
         3.0       1.00      1.00      1.00         5
         4.0       0.75      0.60      0.67         5
         5.0       0.75      0.60      0.67         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.83        30
   macro avg       0.85      0.83      0.83        30
weighted avg       0.85      0.83      0.83        30



3. Evaluation of models not seen in class:
    - **Gaussian Naive Bayes**
    - **Gradient Boosting**
    - **Extra Trees**
    - **Gaussian Process**
    - **Nearest Centroid**

In [ ]:
# imports
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid 
from sklearn.gaussian_process import GaussianProcessClassifier

Gaussian Naive Bayes evaluation:

In [164]:
n_folds = 5
clf = GaussianNB()
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         5
         2.0       0.80      0.80      0.80         5
         3.0       1.00      1.00      1.00         5
         4.0       0.80      0.80      0.80         5
         5.0       0.80      0.80      0.80         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30



Gradient Boosting evaluation:

In [165]:
n_folds = 5
clf = GradientBoostingClassifier(n_estimators=50, max_depth=3, learning_rate=0.1, random_state=42)
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         5
         2.0       0.67      0.80      0.73         5
         3.0       1.00      1.00      1.00         5
         4.0       1.00      0.80      0.89         5
         5.0       0.80      0.80      0.80         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.90        30
   macro avg       0.91      0.90      0.90        30
weighted avg       0.91      0.90      0.90        30



NOTE: An evaluation was attempted with a higher number of decision trees (100), but it overfitted. The same occurred with a lower number of trees (10) and it underfitted. Therefore, an intermediate number of trees (50) was chosen.

Extra Trees evaluation:

In [166]:
n_folds = 5
clf = ExtraTreesClassifier(n_estimators=50, max_depth=5, random_state=42)
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         5
         2.0       0.80      0.80      0.80         5
         3.0       1.00      1.00      1.00         5
         4.0       1.00      1.00      1.00         5
         5.0       0.80      0.80      0.80         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



Gaussian Process evaluation:	

In [167]:
n_folds = 5
clf = GaussianProcessClassifier(random_state=42)
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         5
         2.0       0.50      0.60      0.55         5
         3.0       1.00      1.00      1.00         5
         4.0       0.60      0.60      0.60         5
         5.0       0.75      0.60      0.67         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.80        30
   macro avg       0.81      0.80      0.80        30
weighted avg       0.81      0.80      0.80        30



Nearest Centroid evaluation:

In [168]:
n_folds = 5
clf = NearestCentroid()
y_pred = cross_val_predict(clf, X_scaled, y, cv=n_folds)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         5
         2.0       0.57      0.80      0.67         5
         3.0       1.00      1.00      1.00         5
         4.0       0.60      0.60      0.60         5
         5.0       1.00      0.60      0.75         5
         6.0       1.00      1.00      1.00         5

    accuracy                           0.83        30
   macro avg       0.86      0.83      0.84        30
weighted avg       0.86      0.83      0.84        30



# Classification model optimization

### 1. Model selection:

    - **SVM with radial basis**
    - **Gradient Boosting**

SVM with radial basis hyperparameters:
- **C:** Is the regularization parameter that controls the margin of separation between classes. A higher value allows a narrower margin, while a lower value allows a wider margin.
- **kernel:** Is the type of kernel used to transform the data. In this case, the radial (RBF) kernel is used.
- **gamma:** Is the radial kernel parameter that controls the shape of the decision function. A higher value means the decision function will be more complex, while a lower value means it will be simpler.

Gradient Boosting hyperparameters:
- **n_estimators:** Is the number of decision trees used in the model. A higher value means the model will be more complex, while a lower value means it will be simpler. In this case, a value of 50 was used.
- **max_depth:** Is the maximum depth of each decision tree. A higher value means the model will be more complex, while a lower value means it will be simpler. In this case, a value of 3 was used.
- **learning_rate:** Is the learning rate that controls how much the model adjusts in each iteration. A higher value means the model will adjust faster, while a lower value means it will adjust slower. In this case, a value of 0.1 was used.

### 2. Hyperparameter performance of each model.

SVM with radial basis gamma values performance

In [ ]:
from matplotlib import pyplot as plt

gamma_values = np.logspace(-6, 1, 50)  # From 10^-6 to 10^1
accuracies_rbf = []
n_folds = 5

# Test each gamma value
for i, gamma in enumerate(gamma_values):
    model_rbf = SVC(kernel='rbf', gamma=gamma, C=1.0, random_state=42)
    scores = cross_val_score(model_rbf, X_scaled, y, cv=n_folds, scoring='accuracy')
    accuracy_rbf = scores.mean()
    accuracies_rbf.append(accuracy_rbf)

# Plot average accuracy as a function of gamma
plt.figure(figsize=(12, 6))
plt.plot(gamma_values, accuracies_rbf, marker='o', linestyle='-')
plt.title('Average accuracy with radial SVM as a function of gamma value')
plt.xlabel('Gamma value')
plt.ylabel('Average accuracy')
plt.xscale('log')  # Logarithmic scale for better visualization
plt.grid(True)
plt.tight_layout()
plt.show()

It can be observed that accuracy remains constant around 0.83 in the low range (10^-6 to 10^-2) of gamma, indicating underfitting with very smooth decision boundaries. Accuracy reaches its maximum near gamma = 0.1 with approximately 0.9 accuracy, representing the optimal balance between bias and variance. For gamma values ≥ 1.0, accuracy drops to random classification levels (~0.17 for 6 classes), indicating severe overfitting where the model memorizes individual training points but does not generalize.

Gradient Boosting gamma values performance

In [ ]:
max_depth_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # max_depth values to test
accuracies_gb = []
n_folds = 5

# Test each max_depth value
for i, max_depth in enumerate(max_depth_values):
    model_gb = GradientBoostingClassifier(n_estimators=50, max_depth=max_depth, learning_rate=0.1, random_state=42)
    scores = cross_val_score(model_gb, X_scaled, y, cv=n_folds, scoring='accuracy')
    accuracy_gb = scores.mean()  # Average accuracy across folds

    # Store the accuracy for each max_depth
    accuracies_gb.append(accuracy_gb)

# Plot average accuracy as a function of max_depth
plt.figure(figsize=(12, 6))
plt.plot(max_depth_values, accuracies_gb, marker='o', linestyle='-')
plt.title('Average accuracy with Gradient Boosting as a function of max_depth value')
plt.xlabel('max_depth value')
plt.ylabel('Average accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()

The graph shows the behavior of the Gradient Boosting model performance as a function of the maximum depth of decision trees, where it is observed that the optimal value is at max_depth=2, with an accuracy of around 93%. Being max_depth=3 the value used in the initial model evaluation demonstrates it was a good choice but not the optimal one.

In [ ]:
n_estimators_values = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
accuracies_gb = []
n_folds = 5

# Test each n_estimators value
for i, n_estimators in enumerate(n_estimators_values):
    model_gb = GradientBoostingClassifier(n_estimators=n_estimators, max_depth=3, learning_rate=0.1, random_state=42)
    scores = cross_val_score(model_gb, X_scaled, y, cv=n_folds, scoring='accuracy')
    accuracy_gb = scores.mean()  # Average accuracy across folds

    accuracies_gb.append(accuracy_gb)
    

# Plot average accuracy as a function of n_estimators
plt.figure(figsize=(12, 6))
plt.plot(n_estimators_values, accuracies_gb, marker='o', linestyle='-')
plt.title('Average accuracy with Gradient Boosting as a function of n_estimators value')
plt.xlabel('n_estimators value')
plt.ylabel('Average accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()

The graph shows that the Gradient Boosting model accuracy gradually improves from n_estimators=10 until reaching an optimal point around n_estimators=50-60, where accuracy stabilizes at 90%, confirming that the value of 50 estimators used in the initial evaluation was a good choice as it falls within the optimal range that avoids both underfitting (too few trees) and potential overfitting (too many trees), and that significantly increasing the number of estimators beyond 60-70 does not provide substantial improvements in model performance.

In [ ]:
learning_rate_values = [0.01, 0.05, 0.1, 0.2, 0.5]
accuracies_gb = []
n_folds = 5

# Test each learning_rate value
for i, learning_rate in enumerate(learning_rate_values):
    model_gb = GradientBoostingClassifier(n_estimators=50, max_depth=3, learning_rate=learning_rate, random_state=42)
    scores = cross_val_score(model_gb, X_scaled, y, cv=n_folds, scoring='accuracy')
    accuracy_gb = scores.mean()  # Average accuracy across folds

    accuracies_gb.append(accuracy_gb)
    
# Plot average accuracy as a function of learning_rate
plt.figure(figsize=(12, 6))
plt.plot(learning_rate_values, accuracies_gb, marker='o', linestyle='-')
plt.title('Average accuracy with Gradient Boosting as a function of learning_rate value')
plt.xlabel('learning_rate value')
plt.ylabel('Average accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()

The graph reveals that the optimal learning_rate is in the range of 0.04-0.1, where accuracy reaches 90%, while very low values like 0.02 result in underfitting because the model learns too slowly and fails to converge effectively, and high values like 0.5 can cause overfitting or training instability, validating that the learning_rate=0.1 used in the initial model evaluation was an optimal choice that allows an adequate balance between convergence speed and learning stability.

### 3. Feature selection

SVM with radial basis:

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("-----Optimal selection of features using SelectKBest with SVM Radial-----")

# Feature range
n_features = range(10, 21, 2)  # From 10 to 20, step 2

# Store results
results = []
best_selector = None  # To save the optimal selector

for k in n_features:
    print(f"Evaluating with k={k} features...")
    accuracies = []
    
    for train_index, test_index in skf.split(X_scaled, y):
        # Divide data
        x_train = X_scaled[train_index, :]
        x_test = X_scaled[test_index, :]
        y_train = y[train_index]
        y_test = y[test_index]
        
        # Feature selection
        selector = SelectKBest(score_func=f_classif, k=k)
        x_train_selected = selector.fit_transform(x_train, y_train)
        x_test_selected = selector.transform(x_test)
        
        # Classifier
        clf = SVC(kernel='rbf', gamma=0.1, C=1.0, random_state=42)
        clf.fit(x_train_selected, y_train)
        
        # Predicting
        y_pred = clf.predict(x_test_selected)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
    # Store the average accuracy for this k
    avg_accuracy = np.mean(accuracies)
    results.append((k, avg_accuracy))
    
    # Save the best selector during the loop
    if len(results) == 1 or avg_accuracy > max(result[1] for result in results[:-1]):
        best_selector = selector
    
    print("Average accuracy for k=", k, ":", avg_accuracy)
    
# Optimal number of features (calculate after the loop)
results_array = np.array(results)
opt_index = np.argmax(results_array[:, 1])  # Get index of maximum accuracy
optimal_k = results_array[opt_index, 0]
optimal_accuracy = results_array[opt_index, 1]

# Ensure that best_selector reflects the optimal
if optimal_k != results[opt_index][0]:  # If we need to recreate the selector
    for train_index, test_index in skf.split(X_scaled, y):
        x_train = X_scaled[train_index, :]
        y_train = y[train_index]
        selector = SelectKBest(score_func=f_classif, k=int(optimal_k))
        selector.fit(x_train, y_train)
        best_selector = selector
        break

print("\n--- Final Results ---")
print("Optimal number of features:", int(optimal_k))
print("Optimal accuracy:", optimal_accuracy)
print("Selected features (indices):", np.where(best_selector.get_support())[0])
print("--------------------------------------------------")

Gradient Boosting:

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("-----Optimal selection of features using SelectKBest with Gradient Boosting-----")

n_features = range(10, 21, 2) 

results = []
best_selector = None
best_k = None

for k in n_features:
    print(f"Evaluating with k={k} features...")
    accuracies = []
    
    for train_index, test_index in skf.split(X_scaled, y):
        # Divide data
        x_train = X_scaled[train_index, :]
        x_test = X_scaled[test_index, :]
        y_train = y[train_index]
        y_test = y[test_index]
        
        # Feature selection
        selector = SelectKBest(score_func=f_classif, k=k)
        x_train_selected = selector.fit_transform(x_train, y_train)
        x_test_selected = selector.transform(x_test)
        
        # Classifier
        clf = GradientBoostingClassifier(n_estimators=55, max_depth=2, learning_rate=0.1, random_state=42)
        clf.fit(x_train_selected, y_train)
        
        # Predicting
        y_pred = clf.predict(x_test_selected)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
    # Store the average accuracy for this k
    avg_accuracy = np.mean(accuracies)
    results.append((k, avg_accuracy))
    
    # Save the best k and selector during the loop
    if len(results) == 1 or avg_accuracy > max(result[1] for result in results[:-1]):
        best_k = k
        best_selector = selector
    
    print("Average accuracy for k=", k, ":", avg_accuracy)
    
# Optimal number of features
results_array = np.array(results)
opt_index = np.argmax(results_array[:, 1])  # Get index of maximum accuracy
optimal_k = results_array[opt_index, 0]
optimal_accuracy = results_array[opt_index, 1]

# Ensure that best_selector reflects the optimal
if best_k != optimal_k:
    for train_index, test_index in skf.split(X_scaled, y):
        x_train = X_scaled[train_index, :]
        y_train = y[train_index]
        selector = SelectKBest(score_func=f_classif, k=optimal_k)
        selector.fit(x_train, y_train)
        best_selector = selector
        break

print("\n--- Final Results ---")
print("Optimal number of features:", int(optimal_k))
print("Optimal accuracy:", optimal_accuracy)
print("Selected features (indices):", np.where(best_selector.get_support())[0])
print("--------------------------------------------------")

Selected features:
- **SVM with radial basis:** 
1. Signal mean 
2. Signal median
3. 25th percentile
4. 75th percentile
5. 90th percentile
6. Average power 
7. Velocity standard deviation (1st derivative)
8. Third moment
9. Fourth moment
10. RMS (final)

- **Gradient Boosting:**
1. Signal median
2. Signal variance
3. 25th percentile
4. 75th percentile
5. 90th percentile
6. Average power
7. Velocity standard deviation (1st derivative)
8. Acceleration standard deviation (2nd derivative)
9. Third moment
10. Fourth moment

### Is it possible to reduce the number of features without losing performance in your model?

Yes, it is possible to reduce the number of features, in this case, to 10 without losing performance. In the case of SVM, a notable improvement is achieved (from 0.83 to 0.9667). This validates the effectiveness of SelectKBest for optimizing audio amplitude data.

### 4. Model evaluation and feature selection with nested cross-validation

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

# Inner loop for hyperparameter optimization and k
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Outer loop for evaluating the performance of the optimized model
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) 

print("----- SVM with radial basis with Nested Cross-Validation -----")

# Pipeline: Feature Selection -> SVM Classifier
pipe_svm_nested = Pipeline([('selectkbest', SelectKBest(score_func=f_classif)),('svc', SVC(kernel='rbf', probability=True, random_state=42))])

# Hyperparameter search space (based on previous explorations)
param_grid_svm_nested = {'selectkbest__k': [10, 12, 14],  'svc__C': [1.0, 10.0, 20.0],'svc__gamma': [0.05, 0.1, 0.15]}

# Configure GridSearchCV for the inner loop
grid_search_svm = GridSearchCV(estimator=pipe_svm_nested, param_grid=param_grid_svm_nested, cv=inner_cv, scoring='accuracy', n_jobs=-1) 

y_pred_svm_nested = cross_val_predict(grid_search_svm, X_scaled, y, cv=outer_cv)

print("\nClassification Report for SVM with radial basis (Nested):")
print(classification_report(y, y_pred_svm_nested))
accuracy_svm_nested = accuracy_score(y, y_pred_svm_nested)
print(f"General average accuracy (Nested) - SVM with radial basis: {accuracy_svm_nested:.4f}\n")

# Train GridSearchCV on all X_scaled data to get global parameters and selected features
grid_search_svm.fit(X_scaled, y)
print("Best hyperparameters:")
print(grid_search_svm.best_params_)
print(f"Best accuracy of internal CV - SVM: {grid_search_svm.best_score_:.4f}")

# Get and show the features selected by the best global estimator
best_pipeline_svm = grid_search_svm.best_estimator_  
selected_features_mask_svm = best_pipeline_svm.named_steps['selectkbest'].get_support()
selected_features_indices_svm = np.where(selected_features_mask_svm)[0]
print("Indices of selected features - SVM:")
print(selected_features_indices_svm)
print("--------------------------------------------------\n")

In [ ]:
print("----- Gradient Boosting with Nested Cross-Validation -----")

# Pipeline: Feature Selection -> Gradient Boosting Classifier
pipe_gb_nested = Pipeline([('selectkbest', SelectKBest(score_func=f_classif)),('gb', GradientBoostingClassifier(random_state=42))])

# Hyperparameter search space (based on previous explorations)
param_grid_gb_nested = {'selectkbest__k': [10, 12, 14],'gb__n_estimators': [50, 55, 60], 'gb__max_depth': [2, 3], 'gb__learning_rate': [0.05, 0.1]}

# Configure GridSearchCV for the inner loop
grid_search_gb = GridSearchCV(estimator=pipe_gb_nested, param_grid=param_grid_gb_nested, cv=inner_cv, scoring='accuracy',n_jobs=-1)

y_pred_gb_nested = cross_val_predict(grid_search_gb, X_scaled, y, cv=outer_cv)

print("\nClassification Report for Gradient Boosting (Nested):")
print(classification_report(y, y_pred_gb_nested))
accuracy_gb_nested = accuracy_score(y, y_pred_gb_nested)
print(f"General average accuracy (Nested) - Gradient Boosting: {accuracy_gb_nested:.4f}\n")

grid_search_gb.fit(X_scaled, y)
print("Best hyperparameters - Gradient Boosting:")
print(grid_search_gb.best_params_)
print(f"Best accuracy of internal CV - Gradient Boosting: {grid_search_gb.best_score_:.4f}")

best_pipeline_gb = grid_search_gb.best_estimator_
selected_features_mask_gb = best_pipeline_gb.named_steps['selectkbest'].get_support()
selected_features_indices_gb = np.where(selected_features_mask_gb)[0]
print("Indices of selected features (by the best global estimator) - Gradient Boosting:")
print(selected_features_indices_gb)
print("--------------------------------------------------")

**SVM with radial basis (Nested):**

*   **General average accuracy (Nested):** 0.9333
*   **Best hyperparameters (global):**
    *   `selectkbest__k`: 10
    *   `svc__C`: 20.0
    *   `svc__gamma`: 0.15
*   **Best accuracy of internal CV:** 0.9667
*   **Indices of selected features:** `[ 4 8 10 11 12 15 18 20 22 23]`
*   **Selected features:**
    1.  Index 4: Signal median
    2.  Index 8: Signal variance
    3.  Index 10: 25th percentile (Q1)
    4.  Index 11: 75th percentile (Q3)
    5.  Index 12: 90th percentile
    6.  Index 15: Average power
    7.  Index 18: Velocity standard deviation (1st derivative)
    8.  Index 20: Acceleration standard deviation (2nd derivative)
    9.  Index 22: Third moment
    10. Index 23: Fourth moment

**Gradient Boosting (Nested):**

*   **General average accuracy (Nested):** 0.9333
*   **Best hyperparameters (global):**
    *   `gb__learning_rate`: 0.05
    *   `gb__max_depth`: 2
    *   `gb__n_estimators`: 50
    *   `selectkbest__k`: 12
*   **Best accuracy of internal CV (global):** 0.8667
*   **Indices of selected features:** `[ 0 4 8 10 11 12 14 15 18 20 22 23]`
*   **Selected features:**
    1.  Index 0: Signal mean
    2.  Index 4: Signal median
    3.  Index 8: Signal variance
    4.  Index 10: 25th percentile (Q1)
    5.  Index 11: 75th percentile (Q3)
    6.  Index 12: 90th percentile
    7.  Index 14: Total energy
    8.  Index 15: Average power
    9.  Index 18: Velocity standard deviation (1st derivative)
    10. Index 20: Acceleration standard deviation (2nd derivative)
    11. Index 22: Third moment
    12. Index 23: Fourth moment

# Online classification

In [ ]:
# SINGLE CELL FOR DATA PROCESSING WITH WINDOWS, TRAINING AND SAVING
# ----- 1. LOADING ORIGINAL DATA (experiment_data) -----
actividades = [('sonido_amb', 1), ('conversacion', 2), ('susurro', 3),('grito', 4), ('music', 5), ('aplausos', 6)]
experiment_data = []

if 'experiment_data' not in locals() or len(experiment_data) == 0:
    for actividad_nombre, actividad_id in actividades:
        for i in range(1, 6):
            nombre_archivo = f"../data/{actividad_nombre}_test{i}.csv" 
            try:
                df = pd.read_csv(nombre_archivo)
                if "Sound pressure level (dB)" in df.columns and "Time (s)" in df.columns:
                    # Get time and signal, remove NaNs from dB signal
                    time_original = df["Time (s)"].values
                    signal_original_db = df["Sound pressure level (dB)"].values
                    
                    # Remove rows where dB is NaN AND time too (if both are NaN they are useless)
                    valid_indices_initial = ~np.isnan(signal_original_db)
                    time_valid = time_original[valid_indices_initial]
                    signal_valid_db = signal_original_db[valid_indices_initial]

                    if len(time_valid) < 2 or len(signal_valid_db) < 2:
                        print(f"File {nombre_archivo} with insufficient data after cleaning NaNs.")
                        continue
                        
                    experiment_data.append({'actividad_nombre': actividad_nombre, 
                                            'actividad_id': actividad_id, 
                                            'time': time_valid, 
                                            'db_signal': signal_valid_db,
                                            'nombre_archivo': nombre_archivo})
                else:
                    print(f"Columns 'Time (s)' or 'Sound pressure level (dB)' not found in {nombre_archivo}")
            except FileNotFoundError:
                print(f"File not found: {nombre_archivo}")
            except Exception as e:
                print(f"Error loading {nombre_archivo}: {e}")
    print(f"Loaded {len(experiment_data)} files in 'experiment_data'.")
else:
    print("'experiment_data' already exists. Using version in memory.")

# ----- 2. FUNCTION TO CALCULATE FEATURES 
def calculate_audio_features_notebook(signal_window):
    sig = signal_window; sig = sig[~np.isnan(sig)]; features_list = []
    if len(sig) > 0:
        features_list.append(np.mean(sig)); features_list.append(np.std(sig))
        features_list.append(scipy_stats_notebook.skew(sig)); features_list.append(scipy_stats_notebook.kurtosis(sig))
        features_list.append(np.median(sig)); features_list.append(np.min(sig)); features_list.append(np.max(sig))
        features_list.append(np.ptp(sig)); features_list.append(np.var(sig))
        features_list.append(np.percentile(sig, 10)); features_list.append(np.percentile(sig, 25))
        features_list.append(np.percentile(sig, 75)); features_list.append(np.percentile(sig, 90))
        features_list.append(np.percentile(sig, 75) - np.percentile(sig, 25)) # IQR
        features_list.append(np.sum(sig**2)); features_list.append(np.mean(sig**2)); features_list.append(np.sum(np.abs(sig)))
        if len(sig) > 1:
            velocity = np.diff(sig); features_list.append(np.mean(velocity)); features_list.append(np.std(velocity))
            if len(velocity) > 1:
                acceleration = np.diff(velocity); features_list.append(np.mean(acceleration)); features_list.append(np.std(acceleration))
            else: features_list.extend([0.0, 0.0]) 
        else: features_list.extend([0.0, 0.0, 0.0, 0.0])
        mean_level = np.mean(sig)
        zero_crossings = np.sum(np.diff(np.signbit(sig - mean_level))) if len(sig) > 1 else 0
        features_list.append(zero_crossings / len(sig) if len(sig) > 0 else 0.0) # ZCR
        features_list.append(scipy_stats_notebook.moment(sig, moment=3)); features_list.append(scipy_stats_notebook.moment(sig, moment=4))
        features_list.append(np.mean(np.abs(sig - np.median(sig)))); features_list.append(float(len(sig))) # MAD, Num samples
        std_sig_val = np.std(sig); features_list.append(np.max(sig) / std_sig_val if std_sig_val > 1e-6 else 0.0) # SNR, avoid div by zero
        features_list.append(np.percentile(sig, 90) - np.percentile(sig, 10)) # Dynamic range
        sum_abs_sig = np.sum(np.abs(sig))
        if sum_abs_sig > 1e-6: # Avoid div by zero
            weighted_sum = np.sum(np.abs(sig) * np.arange(len(sig))); features_list.append(weighted_sum / sum_abs_sig) # Centroid
        else: features_list.append(0.0)
        features_list.append(np.mean(np.abs(np.diff(sig))) if len(sig) > 1 else 0.0) # Flux
        rms_val = np.sqrt(np.sum(sig**2) / len(sig)) if len(sig) > 0 else 0.0 # RMS
        features_list.append(rms_val)
    else: features_list.extend([0.0] * 31) # Fill with floating zeros
    
    # Ensure 31 features and that they are floats
    final_features = []
    for feat_val in features_list:
        final_features.append(float(feat_val) if np.isfinite(feat_val) else 0.0)

    if len(final_features) != 31:
        while len(final_features) < 31: final_features.append(0.0)
        final_features = final_features[:31]
    return np.array(final_features)


# ----- 3. FEATURE EXTRACTION WITH SLIDING WINDOWS -----
print("\nExtracting features with sliding windows...")
all_windowed_features = []

# Parameters for windowing and interpolation (same as online script)
target_sampling_rate = 20  # Hz (sampling_rate in online)
window_duration_online = 0.5 # seconds (window_time in online)
window_num_samples_online = int(window_duration_online * target_sampling_rate) # 10 samples
step_duration_processing = 0.25 # seconds (50% overlap)

for entry in experiment_data:
    actividad_id = entry['actividad_id']
    original_time = entry['time']
    original_signal_db = entry['db_signal']
    file_name_debug = entry['nombre_archivo']

    if len(original_time) < 2 or len(np.unique(original_time)) < 2 : # Cannot interpolate
        print(f"  Skipping {file_name_debug}: insufficient time data to interpolate ({len(original_time)} points, {len(np.unique(original_time))} unique).")
        continue

    # Interpolate the COMPLETE signal from the file to a high and regular sampling rate (e.g. 100 Hz)
    # This is to have a consistent base before extracting windows.
    # Make sure the original time is not problematic
    if original_time.max() - original_time.min() <= 0:
        print(f"  Skipping {file_name_debug}: Original time range is zero or negative.")
        continue
        
    internal_high_fs = 100 # High internal rate to resample
    duration_of_recording = original_time[-1] - original_time[0]
    num_samples_high_fs = int(duration_of_recording * internal_high_fs)
    if num_samples_high_fs < 2: 
        print(f"  Skipping {file_name_debug}: Recording duration too short for {internal_high_fs}Hz.")
        continue

    t_high_fs = np.linspace(original_time[0], original_time[-1], num_samples_high_fs, endpoint=False)
    
    try:
        from scipy.interpolate import interp1d
        interp_full_signal = interp1d(original_time, original_signal_db, kind='linear', fill_value="extrapolate", bounds_error=False)
        signal_high_fs = interp_full_signal(t_high_fs)
    except ValueError as e:
        print(f"  Error interpolating complete signal from {file_name_debug}: {e}")
        continue

    # Now apply sliding windows over signal_high_fs
    samples_per_window_high_fs = int(window_duration_online * internal_high_fs) # e.g. 0.5s * 100Hz = 50
    samples_per_step_high_fs = int(step_duration_processing * internal_high_fs) # e.g. 0.25s * 100Hz = 25
    
    num_windows_created = 0
    for i in range(0, len(signal_high_fs) - samples_per_window_high_fs + 1, samples_per_step_high_fs):
        raw_window_high_fs = signal_high_fs[i : i + samples_per_window_high_fs] # High resolution window

        # Interpolate this high resolution window to final resolution (20Hz, 10 samples)
        # Time for this raw_window_high_fs goes from 0 to window_duration_online
        t_raw_for_interp = np.linspace(0, window_duration_online, len(raw_window_high_fs), endpoint=False)
        t_uniform_target = np.linspace(0, window_duration_online, window_num_samples_online, endpoint=False)
        
        if len(t_raw_for_interp) < 2 or len(np.unique(t_raw_for_interp)) < 2: continue

        try:
            interp_func_window = interp1d(t_raw_for_interp, raw_window_high_fs, kind='linear', fill_value="extrapolate", bounds_error=False)
            interpolated_window_for_features = interp_func_window(t_uniform_target)
        except ValueError:
            continue
            
        features_1d = calculate_audio_features_notebook(interpolated_window_for_features)
        all_windowed_features.append(np.concatenate(([actividad_id], features_1d)))
        num_windows_created +=1
    # print(f"  From {file_name_debug}: {num_windows_created} feature windows generated.")


processed_data_final = np.array(all_windowed_features)
if processed_data_final.shape[0] == 0:
    print("ERROR: No features generated. Check windowing logic or input data.")
    # Don't continue if there's no data
else:
    print(f"\nTotal feature windows generated: {processed_data_final.shape[0]}")
    X_original_features = processed_data_final[:, 1:]
    y_labels = processed_data_final[:, 0]

    print(f"Shape of X_original_features (windowed): {X_original_features.shape}")
    print(f"Shape of y_labels (windowed): {y_labels.shape}")

    # ----- 4. FIT AND SAVE StandardScaler -----
    scaler_to_save = StandardScaler()
    X_scaled = scaler_to_save.fit_transform(X_original_features)
    with open('../models/standard_scaler_audio.pkl', 'wb') as f:
        pickle.dump(scaler_to_save, f)
    print("StandardScaler saved to '../models/standard_scaler_audio.pkl'")

    # ----- 5. FIT AND SAVE SelectKBest -----
    k_svm_optimal_global = 10
    feature_selector_svm_to_save = SelectKBest(score_func=f_classif, k=k_svm_optimal_global)
    X_scaled_selected_svm = feature_selector_svm_to_save.fit_transform(X_scaled, y_labels)
    with open('../models/audio_feature_selector_svm.pkl', 'wb') as f:
        pickle.dump(feature_selector_svm_to_save, f)
    print(f"SelectKBest (k={k_svm_optimal_global}) saved to '../models/audio_feature_selector_svm.pkl'")
    # print(f"Selected indices (final): {np.where(feature_selector_svm_to_save.get_support())[0]}")


    # ----- 6. TRAIN AND SAVE FINAL SVM MODEL -----
    C_svm_optimal_global = 20.0
    gamma_svm_optimal_global = 0.15
    final_svm_model_to_save = SVC(kernel='rbf', C=C_svm_optimal_global, gamma=gamma_svm_optimal_global, random_state=42, probability=True)
    final_svm_model_to_save.fit(X_scaled_selected_svm, y_labels)
    with open('../models/final_svm_model_svm.pkl', 'wb') as f:
        pickle.dump(final_svm_model_to_save, f)
    print("Final SVM model saved to '../models/final_svm_model_svm.pkl'")
    print("\nProcess completed! .pkl files ready for online application.")

###  Why the New Processing?

We redid the data processing and model retraining with new window configurations (short and sliding) for a fundamental reason: to make the features we train the model with identical in nature and scale to those the model will see in real-time during online classification.

Previously, training features were calculated over complete recordings (long, 40-60 seconds). In contrast, the online application analyzes audio in very short windows (0.5 seconds). A model trained with data from one "language" (long window features) will not understand well the data presented in another "language" (short window features), resulting in low accuracy.

The new process ensures that:

Absolute Consistency: Both training data and online data are now processed by extracting features from 0.5-second windows.

Representative Training: The model (StandardScaler, SelectKBest, SVC) learns from thousands of examples of these short windows, capturing the variability and relevant patterns for that specific time span.

Higher Online Accuracy: When the online application calculates features from a new 0.5-second window, these are directly comparable to what the model was trained on, allowing much more precise and reliable predictions.